In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.5.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
# enter details of your AML workspace
subscription_id = "YOUR SUBSCRIPTION ID HERE"
resource_group = "rg-churn-pred-proj"
workspace = "churn-pred-proj"

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace)

# Creating a Data Asset
This is the data we have downloaded form hugging face website and will be using for model development.

# List Data Stores

In [4]:
stores = ml_client.datastores.list()
for ds_name in stores:
    print(ds_name.name)

workspaceworkingdirectory
workspaceartifactstore
workspacefilestore
workspaceblobstore


In [5]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import time

my_path = "./data/churn.csv"
# web_path = "https://huggingface.co/datasets/scikit-learn/churn-prediction/blob/main/dataset.csv"
# set the version number of the data asset to the current UTC time
v1 = time.strftime("%Y.%m.%d.%H%M%S", time.gmtime())

churn_data = Data(
    name="telco-churn",
    version=v1,
    description="Churning customers of a telecommunication company",
    path=my_path,
    type=AssetTypes.URI_FILE,
    tags={"source_type": "web", "source": "Hugging Face"},
)

# create data asset
ml_client.data.create_or_update(churn_data)

print(f"Data asset created. Name: {churn_data.name}, version: {churn_data.version}")

Uploading churn.csv (< 1 MB): 100%|██████████| 970k/970k [00:00<00:00, 23.1MB/s]




Data asset created. Name: telco-churn, version: 2023.05.26.132043


Access Data

In [ ]:
%pip install -U azureml-fsspec

Check to see if we have the data or not.

In [6]:
import pandas as pd

# get a handle of the data asset and print the URI
data_asset = ml_client.data.get(name="telco-churn", version=v1)
print(f"Data asset URI: {data_asset.path}")

# read into pandas - note that you will see 2 headers in your data frame - that is ok, for now

df = pd.read_csv(data_asset.path)
df.head()

Data asset URI: azureml://subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourcegroups/rg-churn-pred-proj/workspaces/churn-pred-proj/datastores/workspaceblobstore/paths/LocalUpload/980708968c362ca31f4225d8576b9cab/churn.csv


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Compute Cluster for Pipeline

Once you run the cell, wiat and check under compute to see if the compute has been created successfully before proceeding. You also get a green notification too if you have the default settings on. 

In [7]:
from azure.ai.ml.entities import AmlCompute

cpu_compute_target = "cpu-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        # Name assigned to the compute cluster
        name="cpu-cluster",
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS3_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=4,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=180,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )
    
    print(
    f"AMLCompute with name {cpu_cluster.name} is created, the compute size is {cpu_cluster.size}"
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.begin_create_or_update(cpu_cluster)



Creating a new cpu compute target...
AMLCompute with name cpu-cluster is created, the compute size is STANDARD_DS3_V2


# Pipeline job environment creation
Refresh the file explorer to make sure the directory is created

In [8]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

Creating conda yaml file for in the dependencies directory


In [9]:
%%writefile {dependencies_dir}/conda.yaml
channels:
  - conda-forge
dependencies:
  - python=3.8
  - numpy=1.23.5
  - pip=22.3.1
  - scikit-learn=1.2.2
  - pandas=1.5.3
  - matplotlib=3.7.1
  - imbalanced-learn=0.10.1
  - pip:
      - mlflow==1.26.1
      - azureml-mlflow==1.42.0
      - xgboost==1.7.5
name: churn-env

Writing ./dependencies/conda.yaml


## Create Environment
This can take around 5-10 minutes. Check the environment under the environment tab (custom environments). It changes from running to successful if everything is correct. 


In [10]:
from azure.ai.ml.entities import Environment

custom_env_name = "Churn-Proj-scikit-learn"

pipeline_job_env = Environment(
    name=custom_env_name,
    description="Custom environment for Customer Churn pipeline",
    tags={"scikit-learn": "1.2.2"},
    conda_file=os.path.join(dependencies_dir, "conda.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="0.1.1",
)
pipeline_job_env = ml_client.environments.create_or_update(pipeline_job_env)

print(
    f"Environment with name {pipeline_job_env.name} is registered to workspace, the environment version is {pipeline_job_env.version}"
)

Environment with name Churn-Proj-scikit-learn is registered to workspace, the environment version is 0.1.1


Creating the environment takes around 5-10 minutes depending on the packages. <mark>DO NOT PROCEED BEFORE THE ENVIRONMENT IS MARKED AS SUCCESSFUL. </mark>

# Building Pipeline

We require two components, Each component requires the python script (cells will write this to the appropriate file) and the yaml file.

## Component 1: Data Prep
This envolves missing vlaue recification, dropping some columns, encoding and scaling

In [11]:
import os

# create a folder for the script files
script_folder = 'src'
os.makedirs(script_folder, exist_ok=True)
print(script_folder, 'folder created')

src folder created


prep-data.py script
This is the preparation component of the pipeline, some data cleaning, ecnoding and scaling

In [12]:
%%writefile $script_folder/prep-data.py

# import libraries
import argparse
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


def main(args):
    # read data
    print('Reading data ...')
    df = get_data(args.input_data)

    print('Cleaning data ...')
    cleaned_data = clean_data(df)

    print('Encoding data ...')
    encoded_data = encode_data(cleaned_data)

    print('Normalizing data ...')
    normalized_data = normalize_data(encoded_data)

    output_df = normalized_data.to_csv((Path(args.output_data) / "churn_prepped.csv"), index = False)


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--input_data", dest='input_data',
                        type=str)
    parser.add_argument("--output_data", dest='output_data',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# function that reads the data
def get_data(path):
    df = pd.read_csv(path)

    # Count the rows and print the result
    row_count = (len(df))
    print('Preparing {} rows of data'.format(row_count))

    return df


# function that removes useless values and imputes missing ones
def clean_data(df):
    # Column TotalCharges is a string, have to convert to numeric
    df.TotalCharges = df.TotalCharges.apply([lambda x: float(x) if x!= ' ' else x]) # make float if value exists
    mean = pd.to_numeric(df.TotalCharges, errors='coerce').mean()
    df.TotalCharges = df.TotalCharges.apply([lambda x: mean if x == ' ' else x ]) # replace ' ' with mean of this column

    # Drop useless columns (high cardinality and low correlation clumns - based on the ANOVA test done in EDA)
    df.drop(['customerID', 'gender','PhoneService', 'MultipleLines',
            'InternetService','StreamingTV', 'StreamingMovies'], axis = 1, inplace=True)
    
    return df   

# Function that encodes the data
def encode_data(df):
    cat_cols = ['Partner','Dependents','OnlineSecurity','OnlineBackup',
    	        'DeviceProtection','TechSupport','PaperlessBilling',
                'Contract', 'PaymentMethod'] #categorica columns

    # Encode categorical columns
    ord_enc = OrdinalEncoder()
    df[cat_cols] = ord_enc.fit_transform(df[cat_cols]).copy() 
    # Mapping the target (Churn column)
    lb = LabelEncoder()
    df['Churn'] = lb.fit_transform(df['Churn'])

    return df

# function that normalizes the data
def normalize_data(df):
    # Define Scaler
    mms = MinMaxScaler() # Normalisation using min max scaler
    df['tenure'] = mms.fit_transform(df[['tenure']])
    df['MonthlyCharges'] = mms.fit_transform(df[['MonthlyCharges']])
    df['TotalCharges'] = mms.fit_transform(df[['TotalCharges']])

    return df


# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Writing src/prep-data.py


Preparing YAML file for prep-data

In [13]:
%%writefile prep-data.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: prep_data
display_name: Prepare training data imbalanced sampling
version: 1
type: command
inputs:
  input_data: 
    type: uri_file
outputs:
  output_data:
    type: uri_folder
code: ./src
environment: azureml:Churn-Proj-scikit-learn:0.1.1
command: >-
  python prep-data.py 
  --input_data ${{inputs.input_data}}
  --output_data ${{outputs.output_data}}

Writing prep-data.yml


## Component 2: Train and Evaluate Model
We use mlflow to keep track of the training

train-model.py script

In [14]:
%%writefile $script_folder/train-model.py

import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec

import glob
import argparse

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold

from xgboost import XGBClassifier

def main(args):

    mlflow.start_run()

    # enable autologging
    # mlflow.autolog()

    # read data
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train(args.learning_rate, args.max_depth , X_train, y_train)

    # evaluate model
    evaluate(model, X_test, y_test)

    mlflow.end_run()


# function that reads the data
def get_data(data_path):

    all_files = glob.glob(data_path + "/*.csv")
    df = pd.concat((pd.read_csv(f) for f in all_files), sort=False)
    
    return df

# function that splits the data - uses SMOTE as data unbalanced.
def split_data(df):
    print("Splitting data...")

    oversample = SMOTE(sampling_strategy=1) # same sample size
    f1 = df.iloc[:,:13].values
    t1 = df.iloc[:,13].values
    f1, t1 = oversample.fit_resample(f1, t1)

    X_train, X_test, y_train, y_test = train_test_split(f1, t1, test_size=0.20, random_state=0)

    return X_train, X_test, y_train, y_test


# Function that trains the model, learning rate and max depth are nput args
def train(learning_rate, max_depth, X_train, y_train):
    mlflow.log_param("learning_rate", learning_rate)
    mlflow.log_param("max_depth", max_depth)

    model = XGBClassifier(learning_rate = learning_rate, max_depth = int(max_depth), n_estimators = 1000)

    model.fit(X_train,y_train)
    return model


# Function that evaluates the model
def evaluate(model,X_test,y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))

    params = {
        "Accuracy": acc,
        "AUC": auc,       
        }

    mlflow.log_metrics(params)

    # Confusion Matrix
    cm = ConfusionMatrixDisplay.from_predictions(y_test,y_hat, normalize = 'true', cmap = 'Blues')
    #plt.savefig("Confusion-Matrix.png") 
    mlflow.log_figure(cm.figure_, "Confusion-Matrix.png")

    # plot ROC curve
    roc = RocCurveDisplay.from_predictions(y_test,y_hat)
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    #plt.savefig("ROC-Curve.png") 
    mlflow.log_figure(roc.figure_, "ROC-Curve.png")   

    # Classification Report
    print(classification_report(y_test,y_hat))

    mlflow.log_text(classification_report(y_test,y_hat), "clf_report.txt")


    input_schema = Schema(
        [
            ColSpec("integer", "SeniorCitizen"),
            ColSpec("double", "Partner"),
            ColSpec("double", "Dependents"),
            ColSpec("double", "tenure"),
            ColSpec("double", "OnlineSecurity"),
            ColSpec("double", "OnlineBackup"),
            ColSpec("double", "DeviceProtection"),
            ColSpec("double", "TechSupport"),
            ColSpec("double", "Contract"),
            ColSpec("double", "PaperlessBilling"),
            ColSpec("double", "PaymentMethod"),
            ColSpec("double", "MonthlyCharges"),
            ColSpec("double", "TotalCharges"),
        ]
    )
    output_schema = Schema([ColSpec("integer")])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)




    # Save Model
    mlflow.xgboost.log_model(model, args.model_output, signature=signature)


def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--learning_rate", dest='learning_rate',
                        type=float, default=0.01)
    parser.add_argument("--max_depth", dest='max_depth',
                        type=int, default=3)
    parser.add_argument("--model_output", dest='model_output',
                        type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")


Writing src/train-model.py


train-model.YAML file 

In [15]:
%%writefile train-model.yml
$schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
name: train_model
display_name: Train an XGBoost classifier model
version: 1
type: command
inputs:
  training_data: 
    type: uri_folder
  learning_rate:
    type: number
    default: 0.01
  max_depth:
    type: integer
    default: 3
outputs:
  model_output:
    type: mlflow_model
code: ./src
environment: azureml:Churn-Proj-scikit-learn:0.1.1
command: >-
  python train-model.py 
  --training_data ${{inputs.training_data}} 
  --learning_rate ${{inputs.learning_rate}}
  --max_depth ${{inputs.max_depth}}
  --model_output ${{outputs.model_output}} 

Writing train-model.yml


## Load Components

In [18]:
from azure.ai.ml import load_component
parent_dir = ""

prep_data = load_component(source=parent_dir + "./prep-data.yml")
train_XGBoost = load_component(source=parent_dir + "./train-model.yml")

## Build Pipeline

In [19]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline

@pipeline()
def customer_churn_classification(pipeline_job_input):
    clean_data = prep_data(input_data=pipeline_job_input)
    train_model = train_XGBoost(training_data=clean_data.outputs.output_data)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
    }

pipeline_job = customer_churn_classification(Input(type=AssetTypes.URI_FILE, path= data_asset.path))

In [20]:

print(pipeline_job)

display_name: customer_churn_classification
type: pipeline
inputs:
  pipeline_job_input:
    type: uri_file
    path: azureml://subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourcegroups/rg-churn-pred-proj/workspaces/churn-pred-proj/datastores/workspaceblobstore/paths/LocalUpload/980708968c362ca31f4225d8576b9cab/churn.csv
outputs:
  pipeline_job_transformed_data:
    type: uri_folder
  pipeline_job_trained_model:
    type: mlflow_model
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.pipeline_job_input}}
    outputs:
      output_data: ${{parent.outputs.pipeline_job_transformed_data}}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: prep_data
      version: '1'
      display_name: Prepare training data imbalanced sampling
      type: command
      inputs:
        input_data:
          type: uri_file
      outputs:
        output_data:
          type: uri_folder
   

Update pipeline job. 

In [21]:
# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"
# set pipeline level compute
pipeline_job.settings.default_compute = "cpu-cluster"
# set pipeline level datastore
pipeline_job.settings.default_datastore = "workspaceblobstore"

# print the pipeline job again to review the changes
print(pipeline_job)

display_name: customer_churn_classification
type: pipeline
inputs:
  pipeline_job_input:
    type: uri_file
    path: azureml://subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourcegroups/rg-churn-pred-proj/workspaces/churn-pred-proj/datastores/workspaceblobstore/paths/LocalUpload/980708968c362ca31f4225d8576b9cab/churn.csv
outputs:
  pipeline_job_transformed_data:
    mode: upload
  pipeline_job_trained_model:
    mode: upload
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.pipeline_job_input}}
    outputs:
      output_data: ${{parent.outputs.pipeline_job_transformed_data}}
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: prep_data
      version: '1'
      display_name: Prepare training data imbalanced sampling
      type: command
      inputs:
        input_data:
          type: uri_file
      outputs:
        output_data:
          type: uri_folder
      command

## Submit the pipeline

In [22]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_churn"
)
pipeline_job

Uploading src (0.01 MBs): 100%|██████████| 7155/7155 [00:00<00:00, 196856.91it/s]




Experiment,Name,Type,Status,Details Page
pipeline_churn,olden_sheep_lzm6y4wfmb,pipeline,Preparing,Link to Azure Machine Learning studio


This takes at least 15-20 minutes to run through.
Once the run is completed, you can see the logged metrics, ROC plot, Confusion Matrix, and feature importance plot. 

## Optional:

If you want to register the componets created in the workspace for other users, please run the cell below. Otherwise disregard it.

In [72]:
prep_data = ml_client.components.create_or_update(prep_data)
train_XGBoost = ml_client.components.create_or_update(train_XGBoost)

In [75]:
type(prep_data)

azure.ai.ml.entities._component.command_component.CommandComponent

## Register Model

In [25]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import ModelType


job_name = "e4c67f0e-4090-43e6-9449-c271591d272d"

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="churn_model",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)

Model({'job_name': 'e4c67f0e-4090-43e6-9449-c271591d272d', 'is_anonymous': False, 'auto_increment_version': False, 'name': 'churn_model', 'description': 'Model created from run.', 'tags': {}, 'properties': {}, 'print_as_yaml': True, 'id': '/subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourceGroups/rg-churn-pred-proj/providers/Microsoft.MachineLearningServices/workspaces/churn-pred-proj/models/churn_model/versions/2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci05027a174bf34ac4bd/code/Users/Ramin_Vali/Customer_Churn', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f139607ce80>, 'serialize': <msrest.serialization.Serializer object at 0x7f139607c460>, 'version': '2', 'latest_version': None, 'path': 'azureml://subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourceGroups/rg-churn-pred-proj/workspaces/churn-pred-proj/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.e4c67f0e-4090-43e6-944

In [26]:
model_example = ml_client.models.get(name="churn_model", version="1")
print(model_example)

creation_context:
  created_at: '2023-05-26T14:38:28.269646+00:00'
  created_by: Ramin Vali
  created_by_type: User
  last_modified_at: '2023-05-26T14:38:28.269646+00:00'
  last_modified_by: Ramin Vali
  last_modified_by_type: User
description: Model created from run.
flavors:
  python_function:
    data: model.xgb
    env: conda.yaml
    loader_module: mlflow.xgboost
    python_version: 3.8.16
  xgboost:
    code: ''
    data: model.xgb
    model_class: xgboost.sklearn.XGBClassifier
    xgb_version: 1.7.5
id: azureml:/subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourceGroups/rg-churn-pred-proj/providers/Microsoft.MachineLearningServices/workspaces/churn-pred-proj/models/churn_model/versions/1
job_name: e4c67f0e-4090-43e6-9449-c271591d272d
name: churn_model
path: azureml://subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourceGroups/rg-churn-pred-proj/workspaces/churn-pred-proj/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.e4c67f0e-4090-43e6-9449-c271591d272d/

# Define and create an endpoint

In [50]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

online_endpoint_name = "endpoint-" + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for MLflow customer churn model",
    auth_mode="key",
)

Create the endpoint

In [51]:
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-05261921266135.eastus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-05261921266135.eastus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-05261921266135', 'description': 'Online endpoint for MLflow customer churn model', 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/resourcegroups/rg-churn-pred-proj/providers/microsoft.machinelearningservices/workspaces/churn-pred-proj/onlineendpoints/endpoint-05261921266135', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/733ee2ff-5a9c-42a2-8242-b8a500eb9f53/providers/Microsoft.MachineLearningServices/locations/eastus/mfeOperationsStatus/oe:124b81bd-ce3b-4e78-acb1-ece848a87681:136c51ee-f935-4509-be4e-2bb0ee581032?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/733ee2ff-5a9c-42a2-8242-b8a

This step take some time. Please wait until the notification pops up, or you can see the endpoint created successfully under the endpoints tab.

# Configuring the deployment

We already have our MLflow model to deploy. Given it is an MLflow model, we do not need to supply an environment. Normal models require the model and the environment registered separably. Need to be careful with instance type. I do not have access to all of them because of my tier, the current selection won't work on very large models, but is good for the demonstration purposes.

In [56]:
from azure.ai.ml.entities import Model, ManagedOnlineDeployment
from azure.ai.ml.constants import AssetTypes

# Getting the registered MLflow model. Need to rpvide name and version. These are available under model tab.
model = ml_client.models.get(name='churn_model', version=2)

# Configure a blue deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=online_endpoint_name,
    model=model,
    instance_type="Standard_E2s_v3",
    instance_count=1,
)


# Deploying the model to the online endpoint

In [57]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint endpoint-05261921266135 exists
data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


...............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

HttpResponseError: (None) InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror
Code: None
Message: InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror
Exception Details:	(None) InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror
	The build log is available in the workspace blob store "churnprestoragec3697ee7e" under the path "/azureml/ImageLogs/496008f2-073a-4997-80cc-0df89a887b24/build.log"
	Code: None
	Message: InternalServerError: Internal error. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-internalservererror
	The build log is available in the workspace blob store "churnprestoragec3697ee7e" under the path "/azureml/ImageLogs/496008f2-073a-4997-80cc-0df89a887b24/build.log"

The deployment of the model may take 10-15 minutes. <mark>PLEASE WAIT</mark> for deployment to complete before continuing. 

.................

100% of traffic directed to blue deployment

In [ ]:

# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

Please wait until this update finishes.

In [ ]:
# test the blue deployment with some sample data
response = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name="blue",
    request_file="sample_data.json",
)

if response[1]=='1':
    print("Customer Will Churn")
else:
    print ("Customer Will NOT Churn")

In [ ]:
ml_client.online_endpoints.begin_delete(name=online_endpoint_name)